In [2]:
import pandas as pd
from pathlib import Path
import random
import matplotlib.pyplot as plt
import chardet
from sklearn.metrics import classification_report
import numpy as np

In [3]:
ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data"

SIM_CONST_PATH = DATA_DIR / "generated" / "baseline_constitucion.csv"

sim_df = pd.read_csv(SIM_CONST_PATH, index_col=False)

sim_df


,simulation_id,real_answer,simulated_answer,explanation,edad,sex,region,zone,gse,esc,pol
0,0,Apruebo,Ninguna,"Como persona de la región del Bio-bío, mi prin...",59,hombre,Región del Bio-bío,rural,D,Ed Básica Incompleta,ninguna
1,1,Apruebo,Ninguna,Como persona de 76 años que vive en una zona r...,76,mujer,Región del Maule,rural,D,Ed Básica Incompleta,ninguna
2,2,Apruebo,Ninguna,"En general, la nueva Constitución es un tema i...",45,mujer,Región Metropolitana,urbana,C3,Ed Superior No Universitaria Completa,ninguna
3,3,No sabe,Ninguna,"Como persona de la región del Maule, me siento...",52,mujer,Región del Maule,urbana,C3,Ed Media Completa,no contesta
4,4,Apruebo,Apruebo,Apruebo la nueva Constitución porque considero...,46,mujer,Región Metropolitana,urbana,ABC1,Ed Superior Universitaria Completa,izquierda
...,...,...,...,...,...,...,...,...,...,...,...
1491,1491,Apruebo,Apruebo,Como soy una persona de 79 años que vive en la...,79,hombre,Región de Ñuble,urbana,D,Ed Básica Incompleta,derecha
1492,1492,Apruebo,Ninguna,"En general, me gustaría que la nueva Constituc...",35,mujer,Región de Ñuble,urbana,C3,Ed Superior No Universitaria Incompleta,ninguna
1493,1493,Rechazo,Ninguna,Como persona de una zona rural en la Región de...,48,hombre,Región del Maule,rural,C3,Ed Básica Incompleta,ninguna
1494,1494,Apruebo,Apruebo,"En mi opinión, la nueva Constitución es import...",70,mujer,Región del Maule,rural,D,Ed Básica Incompleta,centro


In [4]:
sim_df.real_answer.value_counts()

real_answer
Apruebo        971
No sabe        298
Rechazo        188
No contesta     39
Name: count, dtype: int64

In [5]:
sim_df.simulated_answer.value_counts()

simulated_answer
Ninguna    868
Apruebo    558
E           51
Rechazo     19
Name: count, dtype: int64

In [6]:
labels_update = { 'E': pd.NA, 'No contesta': 'Ninguna', 'No sabe': 'Ninguna', }

sim_df.real_answer = sim_df.real_answer.replace(labels_update)
sim_df

,simulation_id,real_answer,simulated_answer,explanation,edad,sex,region,zone,gse,esc,pol
0,0,Apruebo,Ninguna,"Como persona de la región del Bio-bío, mi prin...",59,hombre,Región del Bio-bío,rural,D,Ed Básica Incompleta,ninguna
1,1,Apruebo,Ninguna,Como persona de 76 años que vive en una zona r...,76,mujer,Región del Maule,rural,D,Ed Básica Incompleta,ninguna
2,2,Apruebo,Ninguna,"En general, la nueva Constitución es un tema i...",45,mujer,Región Metropolitana,urbana,C3,Ed Superior No Universitaria Completa,ninguna
3,3,Ninguna,Ninguna,"Como persona de la región del Maule, me siento...",52,mujer,Región del Maule,urbana,C3,Ed Media Completa,no contesta
4,4,Apruebo,Apruebo,Apruebo la nueva Constitución porque considero...,46,mujer,Región Metropolitana,urbana,ABC1,Ed Superior Universitaria Completa,izquierda
...,...,...,...,...,...,...,...,...,...,...,...
1491,1491,Apruebo,Apruebo,Como soy una persona de 79 años que vive en la...,79,hombre,Región de Ñuble,urbana,D,Ed Básica Incompleta,derecha
1492,1492,Apruebo,Ninguna,"En general, me gustaría que la nueva Constituc...",35,mujer,Región de Ñuble,urbana,C3,Ed Superior No Universitaria Incompleta,ninguna
1493,1493,Rechazo,Ninguna,Como persona de una zona rural en la Región de...,48,hombre,Región del Maule,rural,C3,Ed Básica Incompleta,ninguna
1494,1494,Apruebo,Apruebo,"En mi opinión, la nueva Constitución es import...",70,mujer,Región del Maule,rural,D,Ed Básica Incompleta,centro


In [7]:
def group_by_age(df):
    df_ = df.copy()
    df_['edad'] = pd.to_numeric(df_['edad'], errors='coerce')

    # Define the bins and labels
    bins = [18, 25, 35, 45, 55, float('inf')]
    labels = ['18-24', '25-34', '35-44', '45-54', '55+']

    # Cut ages into bins
    df_['edad_group'] = pd.cut(df_['edad'], bins=bins, right=False, labels=labels, include_lowest=True)
    return df_

sim_df = group_by_age(sim_df)

In [8]:
def print_metrics(df,subset=None):
    df_ = df.copy()

    categories = df_[subset].unique()

    for cat in categories:
        mask = (df_[subset] == cat)
        print(cat)
        print(classification_report(
            df_[mask].real_answer,
            df_[mask].simulated_answer,
            labels=['Apruebo', 'Rechazo', 'Ninguna'],
            zero_division=np.nan
        ))

In [9]:
print_metrics(sim_df, 'sex')

hombre
              precision    recall  f1-score   support

     Apruebo       0.75      0.50      0.60       412
     Rechazo       0.00      0.00      0.00        78
     Ninguna       0.21      0.62      0.31        96

   micro avg       0.47      0.46      0.46       586
   macro avg       0.32      0.38      0.30       586
weighted avg       0.56      0.46      0.47       586

mujer
              precision    recall  f1-score   support

     Apruebo       0.65      0.33      0.43       559
     Rechazo       0.06      0.01      0.02       110
     Ninguna       0.28      0.68      0.40       241

   micro avg       0.39      0.38      0.39       910
   macro avg       0.33      0.34      0.28       910
weighted avg       0.48      0.38      0.37       910



In [10]:
print_metrics(sim_df, 'edad_group')

55+
              precision    recall  f1-score   support

     Apruebo       0.66      0.43      0.52       359
     Rechazo       0.00      0.00      0.00        63
     Ninguna       0.34      0.66      0.45       177

   micro avg       0.47      0.45      0.46       599
   macro avg       0.34      0.36      0.32       599
weighted avg       0.50      0.45      0.45       599

45-54
              precision    recall  f1-score   support

     Apruebo       0.68      0.38      0.49       175
     Rechazo       0.50      0.03      0.06        31
     Ninguna       0.26      0.68      0.38        62

   micro avg       0.42      0.41      0.41       268
   macro avg       0.48      0.36      0.31       268
weighted avg       0.56      0.41      0.41       268

35-44
              precision    recall  f1-score   support

     Apruebo       0.83      0.39      0.53       150
     Rechazo       0.00      0.00      0.00        37
     Ninguna       0.16      0.79      0.27        28

   m

In [11]:
print_metrics(sim_df,'gse')

D
              precision    recall  f1-score   support

     Apruebo       0.65      0.32      0.43       311
     Rechazo       0.12      0.02      0.03        62
     Ninguna       0.31      0.71      0.43       143

   micro avg       0.41      0.39      0.40       516
   macro avg       0.36      0.35      0.30       516
weighted avg       0.49      0.39      0.38       516

C3
              precision    recall  f1-score   support

     Apruebo       0.73      0.41      0.53       508
     Rechazo       0.00      0.00      0.00        94
     Ninguna       0.23      0.63      0.34       163

   micro avg       0.42      0.41      0.42       765
   macro avg       0.32      0.35      0.29       765
weighted avg       0.54      0.41      0.42       765

ABC1
              precision    recall  f1-score   support

     Apruebo       0.69      0.72      0.71        25
     Rechazo        nan      0.00      0.00        12
     Ninguna       0.25      0.80      0.38         5

    accura

In [16]:
sim_df

string = '{df.gse}'.format(df=sim_df.iloc[0])
string

'D'